In [1]:
# !pip install wilds

     |████████████████████████████████| 126 kB 5.2 MB/s 
     |████████████████████████████████| 503 kB 41.5 MB/s 
     |████████████████████████████████| 4.3 MB 32.4 MB/s 
     |████████████████████████████████| 78 kB 6.9 MB/s 
     |████████████████████████████████| 38.1 MB 1.4 MB/s 
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=485fdca168659b87da49ff0b6a8f772c55ae32cf0b5adda6cd3431eaf8b0a9b4
  Stored in directory: /root/.cache/pip/wheels/d6/64/cd/32819b511a488e4993f2fab909a95330289c3f4e0f6ef4676d
Successfully built littleutils
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Attempting uninstall: pytz
    Found existing installation: pytz 2018.9
    Uninstalling pytz-2018.9:
      Successfully uninstalled pytz-2018.9
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Success

In [67]:
import numpy as np
from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader
import torchvision.transforms as transforms
import torchvision.models as models
from wilds.common.data_loaders import get_eval_loader
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F
from tqdm import tqdm
import random

In [68]:
# Load the full dataset, and download it if necessary
dataset = get_dataset(dataset="camelyon17", download=True)

In [69]:
BATCH_SIZE = 32
FRACTION = 0.33

# Get the training set
train_data = dataset.get_subset(
    "train",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(train_data)) #302436 initially


99804


'\n# Train loop\nfor labeled_batch, unlabeled_batch in zip(train_loader, unlabeled_loader):\n    x, y, metadata = labeled_batch\n    unlabeled_x, unlabeled_metadata = unlabeled_batch\n    ...\n'

In [70]:
# Get the id_val set
id_val_data = dataset.get_subset(
    "id_val",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(id_val_data))

11075


In [71]:
# Get the val set
val_data = dataset.get_subset(
    "val",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(val_data))

11518


In [72]:
# Get the test set
test_data = dataset.get_subset(
    "test",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(test_data))


28068


In [79]:
resnet18_pretrained = models.resnet18(pretrained=True)

In [80]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 5

In [81]:
resnet18_pretrained.fc = nn.Linear(in_features=512, out_features=2, bias=True)
# resnet18_pretrained.load_state_dict(torch.load("resnet18_pretrained_all_grad.pt"))
resnet18_pretrained.to(device)
def remove_classification_head(model):
    modules = list(model.children())[:-1]
    model = nn.Sequential(*modules)
    return model

resnet18_pretrained = remove_classification_head(resnet18_pretrained)

In [82]:
class SiameseNetworkDataset(Dataset):
    def __init__(self,iid_dataset, ood_dataset=None):
        self.data = iid_dataset
        self.ood_data = ood_dataset
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.data)
        
        should_get_same_distribution = random.randint(0,1) # 50% chance
        if should_get_same_distribution == 1:
          img1_tuple = random.choice(self.data) 
                
        else:
          img1_tuple = random.choice(self.ood_data)

        img0 = img0_tuple[0]
        img1 = img1_tuple[0]
        
        return img0, img1, torch.from_numpy(np.array([should_get_same_distribution], dtype=np.float32))
    
    def __len__(self):
        return len(self.data)

In [84]:
train_dataset = SiameseNetworkDataset(train_data, val_data)
id_val_dataset = SiameseNetworkDataset(id_val_data, val_data)

In [86]:
#create the Siamese Neural Network
class SiameseNetwork(nn.Module):

    def __init__(self):
        super(SiameseNetwork, self).__init__()

        # Setting up the Sequential of CNN Layers
        self.embed = resnet18_pretrained

        # Setting up the Fully Connected Layers
        self.fc1 = nn.Sequential(            
            nn.Linear(512,2)
        )
        
    def forward_once(self, x):
        # This function will be called for both images
        # Its output is used to determine the similiarity
        output = self.embed(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        # In this function we pass in both images and obtain both vectors
        # which are returned
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)

        return output1, output2

In [87]:
# Define the Contrastive Loss Function
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
      # Calculate the euclidean distance and calculate the contrastive loss
      euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)

      loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                    (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


      return loss_contrastive

In [88]:
train_loader = DataLoader(train_dataset,
                        shuffle=True,
                        batch_size=BATCH_SIZE)

id_val_loader = DataLoader(id_val_dataset,
                        batch_size=BATCH_SIZE)

net = SiameseNetwork().to(device)

loss_criterion = ContrastiveLoss()

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [89]:
def train_epoch(model, train_dataloader, loss_crt, optimizer, device):
    """
    model: Model object
    train_dataloader: DataLoader over the training dataset
    loss_crt: loss function object
    optimizer: Optimizer object
    device: torch.device('cpu) or torch.device('cuda')

    The function returns:
     - the epoch training loss, which is an average over the individual batch
       losses
    """
    model.train()
    epoch_loss = 0.0

    num_batches = len(train_dataloader)
    # Iterate over batches
    for i, (img0, img1, label) in tqdm(enumerate(train_dataloader, 0)):

        # Send the images and labels to CUDA
        img0, img1, label = img0.to(device), img1.to(device), label.to(device)

        # Zero the gradients
        model.zero_grad()

        # Pass in the two images into the network and obtain two outputs
        output1, output2 = model(img0, img1)

        # Pass the outputs of the networks and label into the loss function
        loss_contrastive = loss_crt(output1, output2, label)

        epoch_loss += loss_contrastive.item()

        # Calculate the backpropagation
        loss_contrastive.backward()

        # Optimize
        optimizer.step()


    epoch_loss = epoch_loss/num_batches
    return epoch_loss

def eval_epoch(model, val_dataloader, loss_crt, device):
    """
    model: Model object
    val_dataloader: DataLoader over the validation dataset
    loss_crt: loss function object
    device: torch.device('cpu) or torch.device('cuda')

    The function returns:
     - the epoch validation loss, which is an average over the individual batch
       losses
    """
    model.eval()
    epoch_loss = 0.0

    num_batches = len(val_dataloader)
    with torch.no_grad():
      # Iterate over batches
      for i, (img0, img1, label) in tqdm(enumerate(val_dataloader, 0)):

          # Send the images and labels to CUDA
          img0, img1, label = img0.to(device), img1.to(device), label.to(device)

          # Pass in the two images into the network and obtain two outputs
          output1, output2 = model(img0, img1)

          # Pass the outputs of the networks and label into the loss function
          loss_contrastive = loss_crt(output1, output2, label)

          epoch_loss += loss_contrastive.item()

    epoch_loss = epoch_loss/num_batches
  
    return epoch_loss

In [90]:
train_losses = []
train_accuracies = []
id_val_losses = []
id_val_accuracies = []
for epoch in range(1, num_epochs+1):
  train_loss = train_epoch(net, train_loader, loss_criterion, optimizer, device)
  val_loss = eval_epoch(net, id_val_loader, loss_criterion, device)
  train_losses.append(train_loss)
  id_val_losses.append(val_loss)
  print('\nEpoch %d'%(epoch))
  print('train loss: %10.8f'%(train_loss))
  print('id_val loss: %10.8f'%(val_loss))

3119it [11:07,  4.67it/s]
347it [01:25,  4.07it/s]



Epoch 1
train loss: 0.18511250
id_val loss: 1.80241119


3119it [09:46,  5.31it/s]
347it [01:11,  4.84it/s]



Epoch 2
train loss: 0.09901252
id_val loss: 1.62022801


3119it [09:21,  5.55it/s]
347it [01:11,  4.89it/s]



Epoch 3
train loss: 0.04757090
id_val loss: 1.93424508


3119it [09:10,  5.67it/s]
347it [01:09,  5.00it/s]



Epoch 4
train loss: 0.04809250
id_val loss: 1.76319959


3119it [09:10,  5.67it/s]
347it [01:11,  4.89it/s]


Epoch 5
train loss: 0.02786437
id_val loss: 1.81552285


In [92]:
import torchvision
from matplotlib.pyplot import imshow
import torchvision.utils
# Grab one image that we are going to test
dataiter = iter(train_loader)
x0, _, label1 = next(dataiter)
dataiter = iter(id_val_loader)

dissimilarity = [[], []]

for i in range(5):
    # Iterate over 5 images and test them with the first image (x0)
    _, x1, label2 = next(dataiter)
    
    output1, output2 = net(x0.cuda(), x1.cuda())

    euclidean_distance = F.pairwise_distance(output1, output2)
    """
    talking on the phone
    """
    for idx in range(BATCH_SIZE):
      # print(label1[idx].item(), label2[idx].item())
      # print(f'Dissimilarity: {euclidean_distance[idx].item():.2f}')
      # print("\n")
      dissimilarity[1 if label1[idx].item() == label2[idx].item() else 0].append(euclidean_distance[idx].item())




In [93]:
import numpy as np
def calc_mean_std(arr):
    arr = np.asarray(arr)
    return arr.mean(), arr.std()


print(calc_mean_std(dissimilarity[0]))
print(calc_mean_std(dissimilarity[1]))

(0.10195220599847811, 0.08368997196062244)
(0.10382272435287368, 0.07654692009587087)


In [100]:
torch.save(net.state_dict(), "contrastive_resnet18_adam.pt")

In [101]:
train_loader = get_train_loader("standard", train_data, batch_size=BATCH_SIZE)
id_val_loader = get_train_loader("standard", id_val_data, batch_size=BATCH_SIZE)
test_loader = get_train_loader("standard", test_data, batch_size=BATCH_SIZE)



In [102]:
x1, label1, _ = next(iter(id_val_loader))

net.eval()
dissimilarity = [[], []]
with torch.no_grad():
    for batch_idx, batch in tqdm(enumerate(train_loader)):
        # Iterate over 5 images and test them with the first image (x0)
        x1, label2, _ = batch
        
        output1, output2 = net(x0.cuda(), x1.cuda())
        try:
            euclidean_distance = F.pairwise_distance(output1, output2)
            
            for idx in range(BATCH_SIZE):
            # print(label1[idx].item(), label2[idx].item())
            # print(f'Dissimilarity: {euclidean_distance[idx].item():.2f}')
                dissimilarity[1 if label1[idx].item() == label2[idx].item() else 0].append(euclidean_distance[idx].item())
        except:
            pass




3119it [05:12,  9.97it/s]


In [103]:
print(calc_mean_std(dissimilarity[0]))
print(calc_mean_std(dissimilarity[1]))

(0.09867890974763098, 0.07807393506476464)
(0.09779296635776634, 0.07629760807766515)


In [104]:
x1, label1, _ = next(iter(test_loader))

net.eval()
dissimilarity = [[], []]
with torch.no_grad():
    for batch_idx, batch in tqdm(enumerate(train_loader)):
        # Iterate over 5 images and test them with the first image (x0)
        x1, label2, _ = batch
        
        output1, output2 = net(x0.cuda(), x1.cuda())
        try:
            euclidean_distance = F.pairwise_distance(output1, output2)
            
            for idx in range(BATCH_SIZE):
            # print(label1[idx].item(), label2[idx].item())
            # print(f'Dissimilarity: {euclidean_distance[idx].item():.2f}')
                dissimilarity[1 if label1[idx].item() == label2[idx].item() else 0].append(euclidean_distance[idx].item())
        except:
            pass

3119it [05:08, 10.11it/s]


In [105]:
print(calc_mean_std(dissimilarity[0]))
print(calc_mean_std(dissimilarity[1]))

(0.09837486262146662, 0.0783391900471739)
(0.0983063083832786, 0.0756569049434798)
